###### 작성자 : 손해원


#### 4월 4일의 삽질
멀티쓰레딩을 위해  ... ! ! ! ! ! 

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from glob import glob
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from tqdm import tqdm
# 멀티스레딩
from multiprocessing.pool import ThreadPool

In [2]:
# 각 업종별 기업 크롤링 함수 

def crawl_(address,name):

    # 크롬드라이버 경로를 확인해 주세요 ! 
    driver = webdriver.Chrome('/Users/user/Jupyter/driver/chromedriver')
    # 네이버 증권홈 -> 국내증시 -> 업종별시세: url에 업종별 number 지정 
    # 각 업종 페이지로 이동 
    driver.get(address)

    # 기본 선택됨 : 거래량, 매수호가, 거래대금, 매도호가, 전일거래량
    # 모두 선택 해제 후 다시 선택 -> 시가총액, per    
    georae_yang = driver.find_element_by_id('option1') # 거래량
    maesoo_hoga = driver.find_element_by_id('option2') # 매수호가
    georae_daegum = driver.find_element_by_id('option3') # 거래대금
    maedo_hoga = driver.find_element_by_id('option8') # 매도호가
    jeon_il_geora = driver.find_element_by_id('option9') # 전일거래량 

    # 선택해제 
    georae_yang.click()
    maesoo_hoga.click()
    georae_daegum.click()
    maedo_hoga.click()
    jeon_il_geora.click()
    
    # 시가총액, per 선택 
    siga_chong = driver.find_element_by_id('option4') # 시가총액
    per = driver.find_element_by_id('option6') # per

    # 선택 
    siga_chong.click()
    per.click()

    # 적용하기 클릭 
    xpath = '''//*[@id="contentarea"]/div[3]/form/div/div/div/a[1]'''
    apply = driver.find_element_by_xpath(xpath)
    apply.click()

    # 기업명, 시가총액, per 가져오기 
    name_list = []
    market_cap_list = []
    per_list = []
    
    # 테이블 
    xpath = '''//*[@id="contentarea"]/div[4]/table/tbody'''
    table = driver.find_element_by_xpath(xpath).text

    # 테이블 안의 이름 영역 
    name_area = driver.find_elements_by_class_name('name_area')
    # 테이블 안의 모든 숫자 -> td class="number" 
    number_area = driver.find_elements_by_class_name('number')

    # 기업명
    name_list = [x.find_element_by_tag_name('a').text for x in name_area]

    # 시가총액, per
    for i in range(len(number_area)):
        # 3번째 인덱스인 시가총액
        if(i%5==3):
            market_cap_list.append((number_area[i].text).replace(',',''))
        # 4번째 인덱스인 per
        elif(i%5==4):
            per_list.append(number_area[i].text)

    # 첫 번째 값이 이상하당 빼버리자 
    market_cap_list = market_cap_list[1:]
    per_list = per_list[1:]

    # 기업 페이지 url
    name_box = driver.find_elements_by_class_name('name_area')
    company_page = [x.find_element_by_tag_name('a').get_attribute('href')  for x in name_box]

    # 각 기업 페이지 접속 -> 종목코드/클래스/상장일 저장
    # 84번 반복합니다 ..! 

    code_list = []
    class_list = []
    reg_day_list = []

    for i in tqdm(range(len(company_page))):
        driver.get(company_page[i])

        # 종목코드 
        code = driver.find_element_by_class_name('code').text
        # print('종목코드는: ', code)
        code_list.append('A'+code)

        # 코스피,코스닥 클래스 
        try:
            driver.find_element_by_class_name('kospi')
            class_ = 'kospi'
        except:
            class_ = 'kosdaq'

        # print('코스피 or 코스닥? :', class_)
        class_list.append(class_)


        # 상장일 
        # url에 종목코드 넣어서 포맷 
        driver.get('https://navercomp.wisereport.co.kr/v2/company/c1020001.aspx?cmp_cd={code}&cn='.format(code=code))

        # 기업개요 클릭
        btn = driver.find_element_by_class_name('line-left')
        btn.click()

        reg_day_long = driver.find_element_by_css_selector('#cTB201 > tbody > tr:nth-child(3) > td.c2.txt').text
        x = reg_day_long.split()
        reg_day = x[-1].replace(')','')
        # print('상장일: ',reg_day)
        reg_day_list.append(reg_day)

    # 종료
    driver.close()


    # 데이터프레임화 합니다 
    df = pd.DataFrame(
        {
             '기업명':name_list
            ,'시가총액':market_cap_list
            ,'per':per_list
            ,'종목코드':code_list
            ,'클래스':class_list
            ,'상장일':reg_day_list
        }
    )
    # csv로 만들어줘 ~~~~~~
    pd.DataFrame(df).to_csv(name+'.csv')


In [5]:
# 업종명 - url 리스트들을 4개로 나눕니당 

def quarter_splitter(lists):
    for company_info in lists:
        try:
            crawl_(company_info[0],company_info[1])
        except:
            continue

In [6]:


# 크롬드라이버 경로를 확인해 주세요 ! 
driver = webdriver.Chrome('/Users/user/Jupyter/driver/chromedriver')
# 네이버 증권홈 -> 국내증시 -> 업종별시세: url에 업종별 number 지정 
# 각 업종 페이지로 이동 
driver.get('https://finance.naver.com/sise/sise_group.nhn?type=upjong')

# 어떤 업종을 선택하시겠습니까 ,,! ,,! ,,! 
a_tags = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('a')
address = [x.get_attribute('href') for x in a_tags]
# 업종명은 
com_name = [x.text for x in a_tags]

# url,업종명 으로 묶인 리스트를 만든다 
addr_name_list=[]
for x,y in zip(address,com_name):
    addr_name_list.append([x,y])
# addr_name_list


# 전체 업종을 4등분해서, 4개의 cpu 프로세서 한개당 하나의 덩어리씩 맡아서 일해라! 라고 던져줍니당 
addr_name1=[]
addr_name2=[]
addr_name3=[]
addr_name4=[]

# 리스트 4등분하고, 한번씩 더 묶어주기 
for i in range(len(addr_name_list)):
    if i%4==0: addr_name1.append(addr_name_list[i])
    elif i%4==1: addr_name2.append(addr_name_list[i])
    elif i%4==2: addr_name3.append(addr_name_list[i])
    elif i%4==3: addr_name4.append(addr_name_list[i])
new_addr_name=[[addr_name1],[addr_name2],[addr_name3],[addr_name4]]

# 일해라 ~~~~~~ 
with ThreadPool(4) as tp:
    tp.starmap(quarter_splitter,new_addr_name)



  0%|          | 0/4 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:02<00:19,  2.76s/it]

 25%|██▌       | 2/8 [00:04<00:15,  2.59s/it]

 38%|███▊      | 3/8 [00:06<00:12,  2.41s/it]

 60%|██████    | 3/5 [00:06<00:04,  2.36s/it]


 50%|█████     | 4/8 [00:09<00:09,  2.34s/it]

 80%|████████  | 4/5 [00:08<00:02,  2.18s/it]


  1%|▏         | 1/69 [00:02<02:58,  2.63s/it]

 62%|██████▎   | 5/8 [00:11<00:06,  2.24s/it]


 75%|███████▌  | 6/8 [00:13<00:04,  2.16s/it]]


 88%|████████▊ | 7/8 [00:15<00:02,  2.23s/it]]


100%|██████████| 8/8 [00:17<00:00,  2.21s/it]]



  0%|          | 0/45 [00:00<?, ?it/s].15s/it]


  2%|▏         | 1/45 [00:03<02:13,  3.02s/it]


  4%|▍         | 2/45 [00:05<01:56,  2.72s/it]


  7%|▋         | 3/45 [00:06<01:40,  2.40s/it]


  0%|          | 0/95 [00:00<?, ?it/s]


  1%|          | 1/95 [00:02<03:43,  2.38s/it]


  2%|▏         | 2/95 [00:04<03:38,  2.34s/it]


  3%|▎         | 3/95 [00:06<03:26,  2.25s/it]


  4%|▍         | 4/95 [00:08<03:08,  2.07s/it]




 68%|██████▊   | 48/71 [01:47<00:53,  2.33s/it]


 22%|██▏       | 32/146 [01:15<04:17,  2.26s/it]

 69%|██████▉   | 49/71 [01:50<00:50,  2.30s/it]


 23%|██▎       | 33/146 [01:17<04:04,  2.16s/it]

100%|██████████| 95/95 [03:46<00:00,  2.39s/it]



 23%|██▎       | 34/146 [01:19<03:46,  2.02s/it]

 72%|███████▏  | 51/71 [01:53<00:41,  2.06s/it]]


 73%|███████▎  | 52/71 [01:56<00:40,  2.15s/it]]

 77%|███████▋  | 92/119 [03:39<01:03,  2.34s/it]


 75%|███████▍  | 53/71 [01:58<00:38,  2.15s/it]]

 78%|███████▊  | 93/119 [03:42<01:02,  2.42s/it]


 76%|███████▌  | 54/71 [02:01<00:38,  2.28s/it]

 79%|███████▉  | 94/119 [03:44<00:58,  2.33s/it]


 10%|█         | 1/10 [00:03<00:29,  3.27s/it]

 80%|███████▉  | 95/119 [03:46<00:56,  2.35s/it]


 79%|███████▉  | 56/71 [02:05<00:33,  2.21s/it][A


 27%|██▋       | 40/146 [01:32<03:51,  2.19s/it]

 80%|████████  | 57/71 [02:07<00:30,  2.16s/it][A


 28%|██▊       | 41/146 [01:34<03:48,  2.17s/it]

 82%|████████▏ | 58/71 [02:09<00:28,  2.22s

  0%|          | 0/25 [00:00<?, ?it/s]2.22s/it]


 95%|█████████▌| 139/146 [04:54<00:14,  2.01s/it]

 53%|█████▎    | 16/30 [00:36<00:30,  2.17s/it]


  4%|▍         | 1/25 [00:02<00:59,  2.49s/it]it]

 57%|█████▋    | 17/30 [00:39<00:28,  2.22s/it]


 97%|█████████▋| 141/146 [04:59<00:10,  2.05s/it]

 60%|██████    | 18/30 [00:41<00:26,  2.19s/it]


 97%|█████████▋| 142/146 [05:01<00:08,  2.04s/it]

 63%|██████▎   | 19/30 [00:43<00:24,  2.19s/it]


 98%|█████████▊| 143/146 [05:03<00:06,  2.05s/it]

 16%|█▌        | 4/25 [00:09<00:49,  2.34s/it]]


 67%|██████▋   | 20/30 [00:45<00:22,  2.24s/it]

 20%|██        | 5/25 [00:11<00:46,  2.32s/it]]


 70%|███████   | 21/30 [00:48<00:20,  2.22s/it]

 24%|██▍       | 6/25 [00:13<00:44,  2.32s/it]]


100%|██████████| 146/146 [05:08<00:00,  2.12s/it]


 28%|██▊       | 7/25 [00:16<00:40,  2.27s/it]]

 32%|███▏      | 8/25 [00:18<00:38,  2.26s/it]]

 36%|███▌      | 9/25 [00:20<00:36,  2.31s/it]]


  0%|          | 0/4 [00:00<?, ?it/s]

 40%|███

 36%|███▌      | 23/64 [00:49<01:27,  2.13s/it]]

 38%|███▊      | 24/64 [00:51<01:25,  2.14s/it]]

 39%|███▉      | 25/64 [00:54<01:28,  2.26s/it]]

 41%|████      | 26/64 [00:56<01:23,  2.19s/it]]

 99%|█████████▉| 99/100 [03:29<00:02,  2.05s/it]

100%|██████████| 100/100 [03:31<00:00,  2.11s/it]


 45%|████▌     | 29/64 [01:03<01:18,  2.24s/it]t]

 47%|████▋     | 30/64 [01:05<01:14,  2.20s/it]t]

 97%|█████████▋| 113/116 [03:57<00:06,  2.07s/it]

 50%|█████     | 32/64 [01:09<01:06,  2.07s/it]t]

 52%|█████▏    | 33/64 [01:11<01:03,  2.04s/it]t]

100%|██████████| 116/116 [04:03<00:00,  2.10s/it]

 59%|█████▉    | 38/64 [01:21<00:54,  2.11s/it][A

 61%|██████    | 39/64 [01:24<00:53,  2.15s/it][A

 62%|██████▎   | 40/64 [01:26<00:51,  2.15s/it][A

 64%|██████▍   | 41/64 [01:28<00:49,  2.14s/it][A

 66%|██████▌   | 42/64 [01:30<00:46,  2.10s/it][A

 67%|██████▋   | 43/64 [01:32<00:44,  2.10s/it][A

 69%|██████▉   | 44/64 [01:34<00:40,  2.03s/it]

 70%|███████   | 45/64 [01:36<00:39, 

 84%|████████▍ | 57/68 [02:11<00:23,  2.16s/it]


 63%|██████▎   | 53/84 [02:02<01:04,  2.08s/it]

 85%|████████▌ | 58/68 [02:13<00:21,  2.12s/it]


 64%|██████▍   | 54/84 [02:04<01:01,  2.07s/it]

  9%|▉         | 7/80 [00:15<02:41,  2.21s/it]


 87%|████████▋ | 59/68 [02:15<00:19,  2.12s/it]

 10%|█         | 8/80 [00:18<02:40,  2.23s/it]


 88%|████████▊ | 60/68 [02:18<00:16,  2.12s/it]

 11%|█▏        | 9/80 [00:20<02:44,  2.32s/it]


  0%|          | 0/11 [00:00<?, ?it/s]

 12%|█▎        | 10/80 [00:22<02:38,  2.27s/it]


  9%|▉         | 1/11 [00:02<00:23,  2.38s/it]


 70%|███████   | 59/84 [02:14<00:49,  1.98s/it]

 18%|█▊        | 2/11 [00:04<00:21,  2.38s/it]


 71%|███████▏  | 60/84 [02:16<00:47,  1.99s/it]

 27%|██▋       | 3/11 [00:06<00:18,  2.32s/it]


 73%|███████▎  | 61/84 [02:18<00:47,  2.06s/it]

 96%|█████████▌| 65/68 [02:28<00:06,  2.16s/it]


 36%|███▋      | 4/11 [00:09<00:16,  2.37s/it]

 97%|█████████▋| 66/68 [02:30<00:04,  2.18s/it]


 99%|█████████▊| 67/68 [0

 40%|████      | 6/15 [00:12<00:18,  2.06s/it]

  4%|▍         | 3/72 [00:06<02:29,  2.17s/it]


 47%|████▋     | 7/15 [00:14<00:16,  2.06s/it][A

  6%|▌         | 4/72 [00:08<02:30,  2.22s/it]


 71%|███████▏  | 5/7 [00:12<00:05,  2.54s/it]

  7%|▋         | 5/72 [00:10<02:19,  2.08s/it]


 86%|████████▌ | 6/7 [00:14<00:02,  2.54s/it]

  8%|▊         | 6/72 [00:12<02:21,  2.15s/it]


 67%|██████▋   | 10/15 [00:20<00:10,  2.05s/it]]

100%|██████████| 7/7 [00:16<00:00,  2.42s/it]



 73%|███████▎  | 11/15 [00:22<00:07,  1.96s/it]]

 11%|█         | 8/72 [00:15<01:59,  1.87s/it]


 80%|████████  | 12/15 [00:24<00:05,  1.98s/it]]

 12%|█▎        | 9/72 [00:18<02:04,  1.98s/it]


  0%|          | 0/1 [00:00<?, ?it/s]

 14%|█▍        | 10/72 [00:20<02:03,  2.00s/it]


 93%|█████████▎| 14/15 [00:28<00:02,  2.09s/it]]

 15%|█▌        | 11/72 [00:22<02:03,  2.03s/it]


100%|██████████| 15/15 [00:30<00:00,  2.04s/it]


 17%|█▋        | 12/72 [00:24<02:02,  2.03s/it]


 18%|█▊        | 26/143 [0

100%|██████████| 3/3 [00:03<00:00,  1.26s/it]]

100%|██████████| 13/13 [00:14<00:00,  1.09s/it]


In [ ]:
### 이거능 ... 업종 넘버 지정해서 멀티스레딩 하기 


# # 업종별 페이지 url 넘버와 저장할 csv파일 이름을 지정합니당 
# food = [38,'식품전체']
# heldevice=[211,'건강관리장비와용품전체']
# elecdevice=[221,'전기제품전체']
# metal = [62,'철강전체']

# # 한번 더 묶어줍니당 
# list_ups = [food,heldevice,elecdevice,metal]


In [ ]:
# 데이터프레임 합치기 ~~~~~~
 #lala = pd.concat([df,df2],axis=1, join='outer')

In [ ]:
# 데이터 전처리 

food = pd.read_csv('./식품전체.csv', encoding='utf-8')
food

In [ ]:
food.drop(['Unnamed: 0'],axis=1, inplace=True)

In [ ]:
food

In [ ]:
food.sort_values(by=['시가총액'], ascending=False, inplace=True)
food

In [ ]:
(food.sort_values(by=['상장일'],ascending=True)).sort_values(by=['시가총액'],ascending=False)